In [ ]:
import warnings
from datetime import time
warnings.filterwarnings("ignore")
import tensorflow as tf
from typing import List
from blindml.frontend.config.task.task import parse_task_capsule
from blindml.frontend.reporting.wit import df_to_examples, create_feature_spec, create_feature_columns, \
    examples_to_array
from witwidget.notebook.visualization import WitConfigBuilder
from witwidget.notebook.visualization import WitWidget

In [2]:
task_file_fp = "/home/maksim/dev_projects/blindml/tests/perovskite_task.jsonnet"
task = parse_task_capsule(task_file_fp)
task.search_for_model()
while not task.get_model_search_update():
    print("no model trained yet")
    time.sleep(2)
model = task.train_best_model()

INFO:  Stopping experiment maxs_experiment_ea88db0ea0f78f5d1c5d9d2885387bfe
INFO:  Stop experiment success.
INFO:  Starting restful server...
INFO:  Successfully started Restful server!
INFO:  Setting local config...
INFO:  Successfully set local config!
INFO:  Starting experiment...


In [5]:
df = task._data_set._df
X_cols, y_col = task._data_set._X_cols, task._data_set._y_col
features_and_labels = X_cols + [y_col]
examples = df_to_examples(df)
feature_spec = create_feature_spec(df, features_and_labels)
create_feature_columns(X_cols, feature_spec);

In [6]:
y_col

'energy_above_hull (meV/atom)'

In [7]:
# takes a proto that's a list of protos
def custom_predict(examples_to_infer: List[tf.train.Example]):
    arr = examples_to_array(examples_to_infer, X_cols)
    return model.predict(arr)

In [9]:
num_datapoints = 1000
tool_height_in_px = 1500

# Load up the test dataset
test_examples = df_to_examples(df[features_and_labels][0:num_datapoints])

# Setup the tool with the test examples and the trained classifier
config_builder = WitConfigBuilder(
    test_examples[:num_datapoints], 
    feature_names=features_and_labels
).set_model_type('regression').set_custom_predict_fn(custom_predict).set_target_feature(y_col)

WitWidget(config_builder)


WitWidget(config={'model_type': 'regression', 'label_vocab': [], 'feature_names': ['covalent radius_AB_avg', '…